# Necessary libraries:
* `gcc` (if you're using windows, it's not that simple).
* `theano`: `!pip install theano`
* `keras`: `!pip install keras`

# Low-level software to implement Neural networks:
* Theano is a python library and an optimizing compiler, which allows to define, optimize and compute mathematical expressions effectively using multidimensional arrays.
    * serves as a low-level backend for many libraries: keras, lasagne, blocks and many others
    * very flexible, though slower then competitors
    * awesome for research
* Tensorflow
    * also quite low-level, but has some of standard deep learning 'elements' built-in
    * considered as a synonym for deep learning by people far from machine learning
    * quite fast and nice if you want to use ML in production. However, mxnet / torch typically not worse
    * better flexibility compared to mxnet / torch / caffe.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import SVG
import numpy

## Import 

In [ ]:
import theano
# next line is a convention
import theano.tensor as T

## Symbolic expessions for tensors
Theano’s strength is in expressing symbolic calculations involving tensors. There are many types of symbolic expressions for tensors:

* scalar
* vector
* matrix
* tensor (3, 4 and more dimensions)

In [ ]:
# create two vectors and one scalar. Those are abstract variables
x_var = T.vector() 
y_var = T.vector()
alpha_var = T.scalar()

#### compute 
$z = x + \alpha y + (\sum x_i, ... ,\sum x_i)^T$

In [ ]:
# define mathematical expression (you can use any function, which you use for numpy arrays). 
z_var = x_var + alpha_var * y_var + T.sum(x_var)

## Compile defined expression
So far we used "symbolic" variables and symbolic expressions defining the expression for computation, but not computing anything. To use the "expression", one should compile it.

We started with defining mathematical function, process of compiling turns this into programmer's function (which is able to efficiently calculate outputs given the input values).

`theano.function` returns a callable object (=function) that will calculate outputs from inputs.

In [ ]:
# input variables, output expessions
inputs = [x_var, y_var, alpha_var]
outputs = z_var
# allow_input_downcast - automatic type casting for input parameters (e.g. float64 -> float32)
compiled_expr = theano.function(inputs, outputs, allow_input_downcast=True)

## Compute compiled expression

In [ ]:
alpha_val = 0.5

# using function with lists:
print "using python lists:"
print compiled_expr([1, 2, 3], [4, 5, 6], alpha_val)
print "\n"

# Or using numpy arrays (should be preferred):
# BTW, that 'float' dtype is casted to second parameter dtype which is float32
print "using numpy arrays:"
print compiled_expr(numpy.arange(10),
                    numpy.linspace(5, 6, 10, dtype='float'), alpha_val)


## Exercises

In [ ]:
# create two vectors and two scalars
x_var = T.vector()
y_var = T.vector()
alpha_var = T.scalar()
beta_var = T.scalar()

# define values for each variable 
x_val = numpy.arange(10)
y_val = numpy.arange(10)
alpha_val = 0.1
beta_val = 0.3

* compute $z = (x_1 + y_1^2, x_2 + y_2^2, ...)^T$: define theano function and evaluate it

* compute $||x||$

* compute $(<x, \alpha y> + <\beta x, y>)^2$, where $ <a , b >  $ is a scalar product

## Names for variables

In [ ]:
# define 
x_var = T.vector(name='x')
y_var = T.vector(name='y')
alpha_var = T.scalar(name='a')

In [ ]:
z_var = alpha_var * x_var * T.log(y_var) 

### `theano.printing.pprint()`
Theano provides the functions `theano.printing.pprint()` and `theano.printing.debugprint()` to print a graph to the terminal before or after compilation. `pprint()` is more compact and math-like, `debugprint()` is more verbose. Theano also provides `pydotprint()` that creates an image of the function. 

In [ ]:
theano.pprint(x_var)

In [ ]:
theano.pprint(z_var)

In [ ]:
compiled_expr = theano.function(inputs=[x_var, y_var, alpha_var], outputs=[z_var], name='function')

In [ ]:
theano.printing.debugprint(compiled_expr)

In [ ]:
theano.printing.pydotprint(compiled_expr, outfile="graph.svg", var_with_name_simple=True, format='svg')
SVG('./graph.svg')

## Gradient — why `theano` matters

* `Theano` can compute derivatives and gradients automatically
* Derivatives are computed symbolically, not numerically: `T.grad` computes symbolic gradients for one or more variables with respect to some variables.

Limitations:
* You can only compute a gradient of a scalar transformation over one or several scalar or vector (or tensor) transformations or inputs.
* A transformation has to have `float32` or `float64` `dtype` throughout the whole computation graph, because derivative over an integer has no mathematical sense



### 1D gradient (derivative)

In [ ]:
x_var = T.scalar(name='x')
squaredx_var = x_var ** 2
squaredx_derivative_var = T.grad(squaredx_var, x_var)

In [ ]:
# result of analytical differentiation:
theano.pprint(squaredx_derivative_var)

In [ ]:
# let's compile it
compiled_function = theano.function([x_var], squaredx_var)
compiled_derivative = theano.function([x_var], squaredx_derivative_var)

In [ ]:
# optimizations were done during compilation
theano.printing.debugprint(compiled_derivative)

In [ ]:
x_val = numpy.linspace(-3, 3, 100)
plt.plot(x_val, map(compiled_function, x_val), label='function')
plt.plot(x_val, map(compiled_derivative, x_val), label='derivative')
plt.legend()

### NDimensional gradient

In [ ]:
x_var = T.vector(name='x')
function_var = T.sum(x_var * x_var)
function_gradient_var = T.grad(function_var, x_var)

In [ ]:
compiled_gradient = theano.function([x_var], function_gradient_var)

In [ ]:
compiled_gradient([1, 2, 4, 0])

In [ ]:
theano.printing.debugprint(compiled_gradient)

## Why T.grad rocks

In [ ]:
def compute_volatility(my_scalar_var, my_vector_var):
    wps = ((my_vector_var + my_scalar_var)**(1 + T.var(my_vector_var)) + \
                  1. / T.arcsinh(my_scalar_var)).mean() / (my_scalar_var**2 + 1) + \
                  0.01 * T.sin(2 * my_scalar_var**1.5) * (T.sum(my_vector_var) * my_scalar_var**2) \
                  * T.exp((my_scalar_var - 4)**2) / (1 + T.exp((my_scalar_var - 4)**2)) * \
                 (1 - (T.exp(-(my_scalar_var-4)**2)) / (1 + T.exp(-(my_scalar_var-4)**2)))**2
    return wps.mean()

# define varibales
my_scalar_var = T.scalar(name='input', dtype='float64')
my_vector_var = T.vector('float64')
volatility_var = compute_volatility(my_scalar_var, my_vector_var)

# define derivatives
derivative_by_scalar_var = T.grad(volatility_var, my_scalar_var)
derivative_by_vector_var = T.grad(volatility_var, my_vector_var)

# compile the function and its derivatives
compiled_fun_function = theano.function([my_scalar_var, my_vector_var], volatility_var)
compiled_fun_derivative_scalar = theano.function([my_scalar_var, my_vector_var], derivative_by_scalar_var)

# Optional exercise on calculus: compute derivatives

In [ ]:
# let's plot for different values of scalar
my_scalar_vals = numpy.linspace(0, 7, 100)
my_vector_val = [1, 2, 3]

plt.plot(my_scalar_vals, [compiled_fun_function(val, my_vector_val) for val in my_scalar_vals], 
         label='function')
plt.plot(my_scalar_vals, [compiled_fun_derivative_scalar(val, my_vector_val) for val in my_scalar_vals],
         label='derivative')
plt.legend(loc='best')

## Shared variables
* The inputs and transformations only exist when function is called
* Shared variables always **stay in memory** like global variables
    * Shared variables are shared between functions where they appear in.
    * Shared variables can be included into a symbolic graph
    * They can be set and evaluated using special methods
        * but they can't change value arbitrarily during symbolic graph computation
   
* Hint: such variables are a perfect place to store network parameters
    * e.g. weights or some metadata

In [ ]:
w_shared = theano.shared(numpy.arange(10, dtype=float), name='weight')

In [ ]:
print w_shared, w_shared.get_value()

In [ ]:
x_var = T.vector('x')
# dotproduct = theano.function([x_var], T.sum(x_var * w_shared))
dotproduct_grad = theano.function([x_var], T.grad(T.sum(x_var * w_shared), x_var))

In [ ]:
dotproduct_grad(numpy.arange(10) * 0.)

In [ ]:
dotproduct_grad(numpy.arange(10) * 10.)

## Matrix

#### Compute $||Ax||^2$:

In [ ]:
x_var = T.vector('x')
A_var = T.matrix('A')
z_var = A_var.dot(x_var)
normAx = theano.function([x_var, A_var], z_var.dot(z_var))
normAx([0, 2], [[1, 1], [1, 1]])

## Exercises

* compile a function which takes an array $x$ with three elements and computes $x_0^3 + \sin{x_1}*\cos{x_2}$. Check it by evaluating at some points

* compute gradient for previous function w.r.t. x

* compute gradient for $||Ax|| + \alpha * ||x||$ w.r.t. x

# Logistic regression 

In [ ]:
# import function to create toy dataset for classification
from sklearn.datasets import make_blobs, make_moons
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
# logistic function (we introduce shortcut sigmoid)
from scipy.special import expit as sigmoid

### generate toy samples

In [ ]:
n_features = 10 # number of features
centers = 2 # number of classes
X, y = make_blobs(n_samples=10000, centers=centers, n_features=n_features, random_state=42, cluster_std=10)
trainX, testX, trainY, testY = train_test_split(X, y, train_size=0.5, random_state=42)

## Logistic regression description:

Implement the regular logistic regression training algorithm

Tips:
* Weights are represented as a `theano` vector
* trainX and trainY are constants, stored in `numpy.arrays`

Exercise:
1. compute probabilities (**use** `T.nnet.sigmoid`):
    $$p_i = \sigma(\sum_k X_{ik} w_k)$$
2. compute logistic loss function
    $$\mathcal{L}=-\sum_i y_i \log{p_i} + (1-y_i)\log{(1 - p_i)}\qquad,$$ where $y \in \{0, 1\}$

3. compute loss function gradient using `theano` 

In [ ]:
# write expression for probabilities
# w_var = 
# p_var = ...

In [ ]:
# write expression for loss
# loss_var = ...

In [ ]:
# compile loss expression, compile gradient expression for loss
# loss_function = theano.function(...)
# loss_grad_function = theano.function(...)

In [ ]:
# very simple check
loss_function(numpy.random.random(n_features))

Now we have expessions for loss and its gradient and we need to use some optimization method

In [ ]:
from scipy.optimize import minimize

In [ ]:
# minimize loss function using its gradient
result = minimize(fun=loss_function, jac=loss_grad_function, x0=numpy.zeros(n_features))

In [ ]:
result

In [ ]:
w_optimal = result['x']

Now predict output of logistic regression for the test sample and compute AUC

In [ ]:
# compute probabilities using numpy
from scipy.special import expit
proba = expit(testX.dot(w_optimal))
print roc_auc_score(testY, proba)

## Last element: updates
* updates are a way of changing shared variables at after function call.
* technically it's a dictionary {shared_variable : a recipe for new value} which is has to be provided when function is compiled

That's how it works:

In [ ]:
# Multiply shared vector by a number and save the product back into shared vector

ones_shared_vector = theano.shared(numpy.ones(10, dtype='float64'))
input_scalar_var = T.scalar('coefficient', dtype='float32')
expression_var = input_scalar_var * ones_shared_vector

inputs = [input_scalar_var]
outputs = [expression_var]  # return vector times scalar

my_updates = {
    ones_shared_vector: expression_var  # and write the result into ones_shared_vector
}

# updates appeared
compute_and_save = theano.function(inputs, outputs, updates=my_updates)

In [ ]:
# initial ones_shared_vector
print "initial shared value:", ones_shared_vector.get_value()

# evaluating the function (ones_shared_vector will be changed)
print "compute_and_save(2) returns", compute_and_save(2)

# evaluate new ones_shared_vector
print "new shared value:", ones_shared_vector.get_value()

# evaluating the function (ones_shared_vector will be changed)
print "compute_and_save(2) returns", compute_and_save(2)

# evaluate new ones_shared_vector
print "new shared value:", ones_shared_vector.get_value()

## Logistic regression with updates in `theano`:

We did it the simplest way, now let's do it *the right way*!

Tips:
* Weights are represented as a shared variable
* X and y are inputs of a function

Compile 2 functions:
* train_function(X, y) — returns an error and computes new values of weights (through updates)
* predict_function(X) — just computes probabilities ("y") given data

In [ ]:
# inputs and shareds
# w_shared = ...
# X_var = ...
# y_var = ...

In [ ]:
# proba_var = ...
# loss_var = ...
# grad_var = ...

# updates = ...

In [ ]:
# train_function = theano.function(...)
# predict_function = theano.function(...)

In [ ]:
# training process
for i in range(15):
    print "loss at iter %i:%.4f" % (i, train_function(trainX, trainY)), 
    print "train auc:", roc_auc_score(trainY, predict_function(trainX)),
    print "test auc:", roc_auc_score(testY, predict_function(testX))

# Neural Network with theano

## NN with one hidden layer
This is a simple NN description with one hidden layer:

Parameters: 

* $W$, $v$

Calculations:

* hidden activations: h = $\sigma$(X.dot(W))
* output = h.dot(v)
* $p_{1}$ = $\sigma$(output)
* $p_{0} = 1 - p_{1}$

## Exercise: 

Write 1-hidden layer NN using theano

In [ ]:
# number of neurons in the hidden layer
n_hidden = 10

# define input variables
# TODO

# define NNs parameters
# TODO

In [ ]:
# define train_function and predict_function
# TODO

In [ ]:
# training process
for i in range(15):
    print "loss at iter %i:%.4f" % (i, train_function(trainX, trainY)), 
    print "train auc:", roc_auc_score(trainY, predict_function(trainX)),
    print "test auc:", roc_auc_score(testY, predict_function(testX))

## NN with two hidden layers
first let's take another dataset

In [ ]:
X, y = make_moons(n_samples=20000, noise=0.1)
trainX, testX, trainY, testY = train_test_split(X, y, train_size=0.5, random_state=42)
n_features = X.shape[1]
plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.05, linewidths=0)

## Exercise

Write 2 hidden layers NN (use code from 1-hidden layer NN)

In [ ]:
# define train_function and predict_function
# TODO

In [ ]:
# training process
for i in range(100):
    print "loss at iter %i:%.4f" % (i, train_function(trainX, trainY)), 
    print "train auc:", roc_auc_score(trainY, predict_function(trainX)),
    print "test auc:", roc_auc_score(testY, predict_function(testX))

## So, what's good about theano?

- symbolic computations with [rich set of operations](http://deeplearning.net/software/theano/library/tensor/index.html)
- very nice correspondence with numpy
- theano operates with symbolic graph to [optimize](http://deeplearning.net/software/theano/optimizations.html) for speed and stability
- code we wrote can be evaluated on GPU 
- loops can be part of a function! Can you differentiate through loops? Theano [can](http://deeplearning.net/software/theano/tutorial/loop.html)
- [optimization for gradients computation](http://deeplearning.net/software/theano/tutorial/gradients.html), like L-operators
- and many other things

## Define special function for NNs

This function simplifies experiments with activations

In [ ]:
def define_NN(layers, activations, learning_rate=0.1):
    X_var = T.matrix('float64')
    y_var = T.vector('float64')
    result = X_var
    
    shared_variables = []
    for dim in range(1, len(layers)):
        w_shared = theano.shared(numpy.random.random(size=(layers[dim - 1], layers[dim])) / 10.)
        result = activations[dim - 1](result.dot(w_shared))
        shared_variables.append(w_shared)
        
    v_shared = theano.shared(numpy.random.random(size=layers[-1]) / 10.)
    shared_variables.append(v_shared)
    
    proba_var = T.nnet.sigmoid(result.dot(v_shared))
    loss_var = -T.mean(y_var * T.log(proba_var) + (1 - y_var) * T.log(1 - proba_var))
        
    updates = {shared: shared - learning_rate * T.grad(loss_var, shared) for shared in shared_variables}

    train_function = theano.function([X_var, y_var], loss_var, updates=updates)
    predict_function = theano.function([X_var], proba_var)
    return train_function, predict_function

In [ ]:
train_function, predict_function = define_NN((n_features, 20, 20), [T.nnet.sigmoid] * 2)

In [ ]:
# training process
for i in range(100):
    print "loss at iter %i:%.4f" % (i, train_function(trainX, trainY)), 
    print "train auc:", roc_auc_score(trainY, predict_function(trainX)),
    print "test auc:", roc_auc_score(testY, predict_function(testX))

## Popular activation functions

* **Sigmoid**:

    $f(x) = \frac{1}  {1+e^{-x}}$


* **ReLU - rectifier linear unit**

    In the context of artificial neural networks, the rectifier is an activation function defined as

    $f(x) = \max(0, x)$

    where x is the input to a neuron. This activation function has been argued to be more biologically plausible than the widely used logistic sigmoid (which is inspired by probability theory; see logistic regression) and its more practical counterpart, the hyperbolic tangent. The rectifier is the most popular activation function for deep neural networks.

    A unit employing the rectifier is also called a rectified linear unit (ReLU).


* **Softplus**
    A smooth approximation to the rectifier is the analytic function

    $f(x) = \ln(1 + e^x)$

    which is called softplus function.

## Let's play with activations: 

Exercise: compare different intermediate activation functions:

* sigmoid (which we used, `T.nnet.sigmoid`)
* leaky ReLU (defined below)
* softplus (`T.nnet.softplus`)

In [ ]:
def LeakyReLU(x):
    return T.switch(x > 0, x, 0.5 * x)

In [ ]:
# Use above define_NN function to play with activations
# TODO

## Feeling the power?

Add multiclassification to the neural network!

You'll need `softmax` activation function for the last layer to compute probabilities.

## NNs available in `hep_ml`
library has some examples of neural networks

In [ ]:
X, y = make_moons(n_samples=2000, noise=0.25, random_state=42)
trainX, testX, trainY, testY = train_test_split(X, y, train_size=0.5, random_state=42)
plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.5, linewidths=0)

In [ ]:
from hep_ml.nnet import PairwiseNeuralNetwork, RBFNeuralNetwork, MLPClassifier

In [ ]:
for models in [PairwiseNeuralNetwork(random_state=42), 
               RBFNeuralNetwork(random_state=42), 
               MLPClassifier(random_state=42)]:
    models.fit(trainX, trainY)
    pred = models.predict_proba(testX)[:, 1]
    print roc_auc_score(testY, pred)

In [ ]:
# just for comparison ...
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=15)
knn_clf.fit(trainX, trainY)
print 'KNN', roc_auc_score(testY, knn_clf.predict_proba(testX)[:, 1])

log_clf = LogisticRegression(C=100)
log_clf.fit(trainX, trainY)
print 'logistic regression', roc_auc_score(testY, log_clf.predict_proba(testX)[:, 1])

rf_clf = RandomForestClassifier(n_estimators=100, max_depth=20)
rf_clf.fit(trainX, trainY)
print 'random forest', roc_auc_score(testY, rf_clf.predict_proba(testX)[:, 1])

gb_clf = GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1)
gb_clf.fit(trainX, trainY)
print 'GB', roc_auc_score(testY, gb_clf.predict_proba(testX)[:, 1])

# Bagging over NN — using sklearn's meta algorithms

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier

In [ ]:
base = RBFNeuralNetwork(layers=(40,))
meta_ada = AdaBoostClassifier(base_estimator=base, n_estimators=10, learning_rate=0.05, random_state=42)
meta_ada.fit(trainX, trainY)

print roc_auc_score(testY, meta_ada.predict_proba(testX)[:, 1])

In [ ]:
base = RBFNeuralNetwork(layers=(40,))
meta_bagging = BaggingClassifier(base_estimator=base, n_estimators=10, random_state=42)
meta_bagging.fit(trainX, trainY)

print roc_auc_score(testY, meta_bagging.predict_proba(testX)[:, 1])

** Nice, this works for simple datasets! Maybe try such tricks for some other data? **

# References
- [theano documentation](http://deeplearning.net/software/theano/)